⚠️ **Ghi chú tối ưu tài nguyên**

Để tránh quá tải VRAM (MX550 ~2GB) và giảm thời gian train, notebook này áp dụng các cắt giảm sau:
- Giảm số nhãn từ **1500 → 500**
- Giảm **batch size từ 128 → 32**
- Giảm số **epochs từ 22 → 4 mỗi fold**
- (Tùy chọn) giảm số **fold từ 5 → 3**

Các thay đổi này giúp giảm đáng kể chi phí tính toán (**≈3–5×**), **đổi lại độ chính xác giảm rõ rệt**: *public score trên Kaggle giảm từ **0.226** (cấu hình gốc) xuống còn **~0.187*** với cấu hình rút gọn.

In [1]:
import os

# --- 1. CẤU HÌNH GỐC (ROOT) ---
# Dùng r'' để Python hiểu đây là đường dẫn Windows (không bị lỗi ký tự đặc biệt)
ROOT = r'E:\CAFA-6-Protein-Function-Prediction'

# --- 2. ĐỊNH NGHĨA CÁC THƯ MỤC CON (Tự động nối đuôi) ---
# os.path.join giúp nối đường dẫn đúng chuẩn cho cả Windows/Linux/Mac
INPUT_DIR  = os.path.join(ROOT, 'input')
OUTPUT_DIR = os.path.join(ROOT, 'output')
MODEL_DIR  = os.path.join(ROOT, 'models')

# Tạo sẵn thư mục output và models nếu chưa có (Tránh lỗi không lưu được file)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"✅ Đã cấu hình ROOT: {ROOT}")
print(f"📂 Input: {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"📂 Models: {MODEL_DIR}")

✅ Đã cấu hình ROOT: E:\CAFA-6-Protein-Function-Prediction
📂 Input: E:\CAFA-6-Protein-Function-Prediction\input
📂 Output: E:\CAFA-6-Protein-Function-Prediction\output
📂 Models: E:\CAFA-6-Protein-Function-Prediction\models


In [2]:
# hàm tính fmax
import numpy as np
def calculate_fmax(preds, targets):
    """
    Tính F-max Score và tìm ngưỡng (threshold) tốt nhất.
    preds: numpy array (N_samples, N_labels) - xác suất dự đoán (sau sigmoid)
    targets: numpy array (N_samples, N_labels) - nhãn thật (0 hoặc 1)
    """
    # Thử các ngưỡng từ 0.01 đến 0.99
    thresholds = np.linspace(0.01, 1.0, 50)
    fmax = 0.0
    best_t = 0.0

    # Chuyển target về int để tính toán
    targets = targets.astype(int)

    for t in thresholds:
        # Áp dụng ngưỡng t để tạo nhãn dự đoán (0 hoặc 1)
        p_cut = (preds >= t).astype(int)

        # Tính TP, FP, FN cho từng mẫu (axis=1)
        tp = (p_cut * targets).sum(axis=1)
        fp = (p_cut * (1 - targets)).sum(axis=1)
        fn = ((1 - p_cut) * targets).sum(axis=1)

        # Tính Precision và Recall cho từng mẫu
        # Tránh chia cho 0: nếu tp+fp=0 -> precision=0; nếu tp+fn=0 -> recall=0
        precision = np.divide(tp, (tp + fp), out=np.zeros_like(tp, dtype=float), where=(tp + fp) != 0)
        recall = np.divide(tp, (tp + fn), out=np.zeros_like(tp, dtype=float), where=(tp + fn) != 0)

        # Tính F1 trung bình trên toàn bộ tập mẫu (Average F1)
        p_avg = precision.mean()
        r_avg = recall.mean()
        
        if (p_avg + r_avg) > 0:
            f1 = 2 * p_avg * r_avg / (p_avg + r_avg)
        else:
            f1 = 0

        if f1 > fmax:
            fmax = f1
            best_t = t

    return fmax, best_t

In [3]:
# Cell 1: Import & Config & Seeding
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import gc
import random
import os

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"Set Global Seed: {seed}")

SEED = 42
seed_everything(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

INPUT_EMBED_DIR = f"{INPUT_DIR}/cafa6-t5-embeddings"

CONFIG = {
    #"num_labels": 1500,
    "num_labels": 500,
    #"batch_size": 128,
    "batch_size": 32,
    "lr": 0.0007,      
    #"epochs": 22,    
    "epochs": 4,
    "seed": SEED,
    "paths": {
        "train_embeds": f"{INPUT_EMBED_DIR}/train_embeddings_esm2.npy",
        "train_ids":    f"{INPUT_EMBED_DIR}/train_ids_esm2.npy",
        "test_embeds":  f"{INPUT_EMBED_DIR}/test_embeddings_esm2.npy",
        "test_ids":     f"{INPUT_EMBED_DIR}/test_ids_esm2.npy",
        "train_terms":  f"{INPUT_DIR}/cafa-6-protein-function-prediction/Train/train_terms.tsv"
    }
}

Set Global Seed: 42
Using device: cuda


In [4]:
# Cell 2: Load Embeddings
print("Loading embeddings from disk...")

# Load Train
train_embeds = np.load(CONFIG['paths']['train_embeds'])
train_ids = np.load(CONFIG['paths']['train_ids'], allow_pickle=True)

# Load Test
test_embeds = np.load(CONFIG['paths']['test_embeds'])
test_ids = np.load(CONFIG['paths']['test_ids'], allow_pickle=True)

# Tự động cập nhật kích thước đầu vào (Input Dimension)
# Giúp code chạy đúng dù vector là T5 (1024) hay ESM2 (1280/2560)
EMBED_DIM = train_embeds.shape[1]

print(f"--- Data Loaded ---")
print(f"Train shape: {train_embeds.shape}")
print(f"Test shape:  {test_embeds.shape}")
print(f"Embedding Dimension detected: {EMBED_DIM}")

Loading embeddings from disk...
--- Data Loaded ---
Train shape: (82404, 1280)
Test shape:  (224309, 1280)
Embedding Dimension detected: 1280


In [5]:
# Cell 3: Prepare Targets (SỬA LỖI ĐỊNH DẠNG ID: sp|ID|NAME -> ID)
import gc

print("--- Processing Targets & Fixing ID Format ---")

# 1. Load dữ liệu
train_terms = pd.read_csv(CONFIG['paths']['train_terms'], sep="\t")
train_ids = np.load(CONFIG['paths']['train_ids'], allow_pickle=True)

# --- SỬA LỖI LOGIC TẠI ĐÂY ---
def clean_id(pid):
    # Chuyển bytes sang string nếu cần
    if isinstance(pid, bytes):
        pid = pid.decode('utf-8')
    pid_str = str(pid).strip()
    
    # Logic tách chuỗi: "sp|A0A0C5B5G6|MOTSC_HUMAN" -> lấy "A0A0C5B5G6"
    # Tách bằng dấu gạch đứng '|' và lấy phần tử thứ 2 (index 1)
    parts = pid_str.split('|')
    if len(parts) > 1:
        return parts[1] # Lấy mã ở giữa
    return pid_str # Trả về nguyên gốc nếu không tìm thấy dấu |

# Áp dụng hàm sửa lỗi
train_ids_clean = [clean_id(pid) for pid in train_ids]

print(f"[DEBUG] ID gốc: {train_ids[0]}")
print(f"[DEBUG] ID sau khi sửa: {train_ids_clean[0]}") # Mong đợi: A0A0C5B5G6

# Tạo map
id_map = {pid: i for i, pid in enumerate(train_ids_clean)}

# 2. Chọn Top Labels
top_terms = train_terms['term'].value_counts().index[:CONFIG['num_labels']]
term_to_idx = {term: i for i, term in enumerate(top_terms)}

# 3. Tạo ma trận Targets
num_samples = len(train_ids)
labels_matrix = np.zeros((num_samples, CONFIG['num_labels']), dtype=np.float32)

# Lọc dữ liệu
train_terms['EntryID'] = train_terms['EntryID'].astype(str).str.strip()
filtered_terms = train_terms[
    (train_terms['EntryID'].isin(id_map)) & 
    (train_terms['term'].isin(top_terms))
]

print(f"Số lượng dòng khớp được: {len(filtered_terms)}")

if len(filtered_terms) == 0:
    print("Error")
else:
    print("Đang tạo ma trận nhãn...")
    for pid, term in tqdm(zip(filtered_terms['EntryID'], filtered_terms['term']), total=len(filtered_terms)):
        row_idx = id_map[pid]
        col_idx = term_to_idx[term]
        labels_matrix[row_idx, col_idx] = 1.0
        
    print(f"THÀNH CÔNG! Tổng số nhãn dương: {labels_matrix.sum()}")

del train_terms, filtered_terms, train_ids_clean
gc.collect()

--- Processing Targets & Fixing ID Format ---
[DEBUG] ID gốc: sp|A0A0C5B5G6|MOTSC_HUMAN
[DEBUG] ID sau khi sửa: A0A0C5B5G6
Số lượng dòng khớp được: 264886
Đang tạo ma trận nhãn...


100%|██████████| 264886/264886 [00:00<00:00, 1039530.16it/s]


THÀNH CÔNG! Tổng số nhãn dương: 264886.0


37

In [6]:
# Cell 4: Enhanced Hybrid Model
class ProteinDataset(Dataset):
    def __init__(self, embeddings, targets=None, ids=None):
        self.embeddings = embeddings
        self.targets = targets
        self.ids = ids
        
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        embed = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return embed, target
        return embed, self.ids[idx]

class ResidualCNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dropout=0.1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.act = nn.GELU() # Đổi sang GELU mượt hơn
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(dropout)
        
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        residual = self.shortcut(x)
        out = self.act(self.bn1(self.conv1(x)))
        out = self.dropout(self.bn2(self.conv2(out)))
        out += residual
        return self.act(out)

class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        b, c = x.size()
        y = x.view(b, c, 1)
        y = self.avg_pool(y).view(b, c)
        y = self.fc(y)
        return x * y

class DenseResBlock(nn.Module):
    def __init__(self, features, dropout_rate=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(features, features * 2), # Expand width
            nn.BatchNorm1d(features * 2),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(features * 2, features), # Project back
            nn.BatchNorm1d(features),
            nn.GELU(),
            nn.Dropout(dropout_rate)
        )
        self.se = SEBlock(features)
        
    def forward(self, x):
        return self.se(self.block(x)) + x # Residual connection

class HybridSystem(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        
        # 1. Nhánh Residual CNN (Sâu hơn, giữ feature tốt hơn)
        self.cnn_branch = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=7, padding=3),
            nn.BatchNorm1d(64),
            nn.GELU(),
            ResidualCNNBlock(64, 128, kernel_size=5),
            nn.MaxPool1d(2),
            ResidualCNNBlock(128, 256, kernel_size=3),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear(256 * (input_dim // 4), 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.3)
        )
        
        # 2. Nhánh Wide ResNet (Mở rộng chiều ngang)
        self.resnet_branch = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            DenseResBlock(1024, dropout_rate=0.2),
            DenseResBlock(1024, dropout_rate=0.2),
            DenseResBlock(1024, dropout_rate=0.2), # Thêm 1 block nữa
        )
        
        # 3. Fusion
        self.fusion = nn.Sequential(
            nn.Linear(1024 + 1024, 1536),
            nn.BatchNorm1d(1536),
            nn.GELU(),
            nn.Dropout(0.3)
        )
        
        # Multi-Sample Dropout
        self.dropouts = nn.ModuleList([nn.Dropout(0.4) for _ in range(5)])
        self.fc = nn.Linear(1536, num_classes)

    def forward(self, x):
        # CNN Branch
        x_cnn = x.unsqueeze(1)
        out_cnn = self.cnn_branch(x_cnn)
        
        # ResNet Branch
        out_res = self.resnet_branch(x)
        
        # Fusion
        combined = torch.cat([out_cnn, out_res], dim=1)
        features = self.fusion(combined)
        
        # Output
        output = torch.zeros(features.size(0), self.fc.out_features).to(features.device)
        for dropout in self.dropouts:
            output += self.fc(dropout(features))
        return output / len(self.dropouts)

In [7]:
# Cell 5: Training with Asymmetric Loss
from sklearn.model_selection import KFold
from torch.optim.lr_scheduler import OneCycleLR
import torch.nn.utils as utils

# --- ASYMMETRIC LOSS ---
class AsymmetricLossOptimized(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False):
        super(AsymmetricLossOptimized, self).__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

    def forward(self, x, y):
        # x: logits, y: targets (0 or 1)
        
        # Tính sigmoid probabilities
        x_sigmoid = torch.sigmoid(x)
        x_sigmoid_pos = x_sigmoid
        x_sigmoid_neg = 1 - x_sigmoid

        # Asymmetric Clipping 
        if self.clip is not None and self.clip > 0:
            x_sigmoid_neg = (x_sigmoid_neg + self.clip).clamp(max=1)

        # Basic Cross Entropy parts
        loss_pos = y * torch.log(x_sigmoid_pos.clamp(min=self.eps))
        loss_neg = (1 - y) * torch.log(x_sigmoid_neg.clamp(min=self.eps))

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            
            pt0 = x_sigmoid_pos * y
            pt1 = x_sigmoid_neg * (1 - y)  # pt = p if t=1 else 1-p
            pt = pt0 + pt1

            one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)

            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            
            loss_pos *= one_sided_w
            loss_neg *= one_sided_w

        return -torch.mean(loss_pos + loss_neg)

# --- CONFIG TRAINING ---
N_FOLDS = 5
#EPOCHS_PER_FOLD = 20
EPOCHS_PER_FOLD = CONFIG['epochs']
#LR = 0.001
LR = CONFIG['lr']
#BATCH_SIZE = 128
BATCH_SIZE = CONFIG['batch_size']
WEIGHT_DECAY = 1e-2 

kfold = KFold(n_splits=N_FOLDS, shuffle=True, random_state=CONFIG['seed'])
print(f"\n--- START TRAINING (ASYMMETRIC LOSS + RESIDUAL MODEL) ---")

fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_embeds, labels_matrix)):
    print(f"\n>>> FOLD {fold+1}/{N_FOLDS}")
    
    # Load Data
    X_train, X_val = train_embeds[train_idx], train_embeds[val_idx]
    y_train, y_val = labels_matrix[train_idx], labels_matrix[val_idx]
    
    train_dataset = ProteinDataset(X_train, y_train)
    val_dataset = ProteinDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=0)
    
    # Init Model
    model = HybridSystem(input_dim=EMBED_DIM, num_classes=CONFIG['num_labels']).to(DEVICE)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    
    criterion = AsymmetricLossOptimized(gamma_neg=4, gamma_pos=1, clip=0.05)
    
    scheduler = OneCycleLR(optimizer, max_lr=LR, steps_per_epoch=len(train_loader), 
                           epochs=EPOCHS_PER_FOLD, pct_start=0.3)
    
    best_loss = float('inf')
    
    for epoch in range(EPOCHS_PER_FOLD):
        model.train()
        train_loss = 0
        for embeds, targets in tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False):
            embeds, targets = embeds.to(DEVICE), targets.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(embeds)
            loss = criterion(outputs, targets)
            loss.backward()
            
            # Gradient Clipping (Vẫn giữ để an toàn)
            utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        val_preds_all = []
        val_targets_all = []
        with torch.no_grad():
            for embeds, targets in val_loader:
                embeds, targets = embeds.to(DEVICE), targets.to(DEVICE)
                outputs = model(embeds)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                val_preds_all.append(torch.sigmoid(outputs).detach().cpu().numpy())
                val_targets_all.append(targets.detach().cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        
        # --- THÊM ĐOẠN TÍNH F-MAX NÀY ---
        # Gộp các batch lại
        val_preds_concat = np.concatenate(val_preds_all, axis=0)
        val_targets_concat = np.concatenate(val_targets_all, axis=0)
        
        # Gọi hàm tính
        val_fmax, val_best_t = calculate_fmax(val_preds_concat, val_targets_concat)
        # --------------------------------

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), f"model_fold_{fold}.pth")
            # --- SỬA PRINT CÓ NEW BEST ---
            print(f"Ep {epoch+1}: Loss={avg_val_loss:.5f} | F-max={val_fmax:.4f} (t={val_best_t:.2f}) ⭐ NEW BEST")
        else:
            # --- SỬA PRINT THƯỜNG ---
            print(f"Ep {epoch+1}: Loss={avg_val_loss:.5f} | F-max={val_fmax:.4f} (t={val_best_t:.2f})")
            
    # --- SỬA DÒNG IN KẾT QUẢ FOLD ---
    print(f"Fold {fold+1} Best ASL Loss: {best_loss:.5f} (Last F-max: {val_fmax:.4f})")


--- START TRAINING (ASYMMETRIC LOSS + RESIDUAL MODEL) ---

>>> FOLD 1/5


KeyboardInterrupt: 

In [ ]:
# Cell 6: Ensemble Prediction (Hybrid System)
import gc
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm

print("\n--- Generating Predictions (Hybrid System) ---")

# 1. Khôi phục mapping
print("Re-creating index mapping...")
df_terms = pd.read_csv(CONFIG['paths']['train_terms'], sep="\t")
top_terms = df_terms['term'].value_counts().index[:CONFIG['num_labels']]
idx_to_term = {i: term for i, term in enumerate(top_terms)}
del df_terms, top_terms
gc.collect()

# 2. Setup Data Loader
BATCH_SIZE = 256
test_dataset_final = ProteinDataset(test_embeds, ids=test_ids)
test_loader_final = DataLoader(test_dataset_final, batch_size=BATCH_SIZE, shuffle=False)

# 3. Load Models
models = []
for fold in range(N_FOLDS):
    # Dùng đúng class HybridSystem
    model = HybridSystem(input_dim=EMBED_DIM, num_classes=CONFIG['num_labels']).to(DEVICE)
    model.load_state_dict(torch.load(f"model_fold_{fold}.pth"))
    model.eval()
    models.append(model)
print(f"Loaded {len(models)} Hybrid models.")

# 4. Prediction
THRESHOLD = 0.02
TOP_K = 75       
output_file = f"{OUTPUT_DIR}\submission_hybrid.tsv"

print(f"Inference... Threshold={THRESHOLD}, Top-K={TOP_K}")

with open(output_file, 'w') as f:
    with torch.no_grad():
        for step, (embeds, batch_ids) in enumerate(tqdm(test_loader_final)):
            embeds = embeds.to(DEVICE)
            
            avg_probs = None
            for model in models:
                logits = model(embeds)
                probs = torch.sigmoid(logits)
                if avg_probs is None: avg_probs = probs
                else: avg_probs += probs
            
            avg_probs /= len(models)
            avg_probs = avg_probs.cpu().numpy()
            
            batch_lines = []
            for i, pid in enumerate(batch_ids):
                row_probs = avg_probs[i]
                idx_candidates = np.where(row_probs > THRESHOLD)[0]
                
                if len(idx_candidates) > TOP_K:
                    candidate_probs = row_probs[idx_candidates]
                    final_indices = idx_candidates[np.argsort(candidate_probs)[-TOP_K:]]
                else:
                    final_indices = idx_candidates
                
                for idx in final_indices:
                    batch_lines.append(f"{pid}\t{idx_to_term[idx]}\t{row_probs[idx]:.3f}\n")
            
            f.writelines(batch_lines)
            del batch_lines, avg_probs, embeds
            if step % 50 == 0: gc.collect()

print(f"✅ Submission saved to {output_file} successfully!")